In [7]:
# 49035114000 (center)
# 49035980000 (airport)
# 49035110106 (ski)
# 49035101402 (U of U)

In [ ]:
# ============================================================
# FAST + ROBUST VERSION
# Build OD-level annual statistics for Complete Trips
# OD defined strictly by Census Tract (GEOID + shapefile)
# Population-level statistics (NOT sampled)
# ============================================================

import pandas as pd
import numpy as np
import geopandas as gpd
import glob
import json
from datetime import datetime
import pygeohash as pgh


# =========================
# CONFIG
# =========================

BASE_DIR = "C:/Users/rli04/Villanova University/Complete-trip-coordinate - Documents/General"
PARQUET_DIR = f"{BASE_DIR}/Salt_Lake/delivery"

TRACT_SHP = (
    f"{BASE_DIR}/Manuscript/Figure/Visualization-RL/"
    f"2-OD patterns by census track/six_counties_track.shp"
)

ORIG_TRACT = "49035110106"
DEST_TRACT = "49035114000"
# 49035114000 (center)
# 49035980000 (airport)
# 49035110106 (ski)
# 49035101402 (U of U)
MONTHS = [
    "Jan"
    # ,"Feb","Mar","Apr","May","Jun",
    # "Jul","Aug","Sep","Oct","Nov","Dec"
]

OUTPUT_STATS_JSON = f"{ORIG_TRACT}_to_{DEST_TRACT}.stats.json"

USE_COLS = [
    "linked_trip_id",
    "travel_mode",
    "local_datetime_start",
    "local_datetime_end",
    "geohash7_orig",
    "geohash7_dest"
]


# =========================
# 0️⃣ LOAD TRACTS
# =========================

tracts = gpd.read_file(TRACT_SHP).to_crs("EPSG:4326")
tracts["GEOID"] = tracts["GEOID"].astype(str)

orig_poly = tracts.loc[tracts["GEOID"] == ORIG_TRACT, ["geometry"]].copy()
dest_poly = tracts.loc[tracts["GEOID"] == DEST_TRACT, ["geometry"]].copy()

if orig_poly.empty or dest_poly.empty:
    raise ValueError("Origin/Destination tract GEOID not found in shapefile.")


# =========================
# 1️⃣ LOAD YEARLY DATA
# =========================

dfs = []
for m in MONTHS:
    files = glob.glob(f"{PARQUET_DIR}/Salt_Lake-{m}-2020/*.snappy.parquet")
    if not files:
        continue
    dfs.append(pd.concat(
        [pd.read_parquet(f, columns=USE_COLS) for f in files],
        ignore_index=True
    ))

if not dfs:
    raise ValueError("No parquet files found. Check PARQUET_DIR and MONTHS.")

df = pd.concat(dfs, ignore_index=True)

df["local_datetime_start"] = pd.to_datetime(df["local_datetime_start"], errors="coerce")
df["local_datetime_end"]   = pd.to_datetime(df["local_datetime_end"], errors="coerce")

df = df[df["local_datetime_end"] > df["local_datetime_start"]]

df = df[df["geohash7_orig"].notna() & df["geohash7_dest"].notna()]
df["geohash7_orig"] = df["geohash7_orig"].astype(str)
df["geohash7_dest"] = df["geohash7_dest"].astype(str)


# =========================
# 2️⃣ FAST GEOHASH DECODE (OPTIMIZED: ONLY FIRST/last needed)
# =========================

def _decode_unique_geohash(series):
    uniq = series.unique()
    mapping = {}
    for gh in uniq:
        try:
            lat, lon = pgh.decode(gh)
            mapping[gh] = (lat, lon)
        except Exception:
            mapping[gh] = (np.nan, np.nan)
    return series.map(mapping).apply(pd.Series).rename(columns={0: "lat", 1: "lon"})


# -------------------------
# OPT 1: avoid full-table geohash decode & avoid full sort
# We first compute first/last row indices per linked_trip_id
# using vectorized idxmin/idxmax on timestamps
# -------------------------

# drop rows with invalid times early (helps groupby idx ops)
df = df[df["local_datetime_start"].notna() & df["local_datetime_end"].notna()]

idx_first = df.groupby("linked_trip_id")["local_datetime_start"].idxmin()
idx_last  = df.groupby("linked_trip_id")["local_datetime_end"].idxmax()

first_trips = df.loc[idx_first, ["linked_trip_id", "geohash7_orig"]].copy()
last_trips  = df.loc[idx_last,  ["linked_trip_id", "geohash7_dest"]].copy()

# decode only needed geohashes (huge speedup)
o_latlon = _decode_unique_geohash(first_trips["geohash7_orig"])
d_latlon = _decode_unique_geohash(last_trips["geohash7_dest"])

first_trips["orig_lat"] = o_latlon["lat"]
first_trips["orig_lon"] = o_latlon["lon"]
last_trips["dest_lat"]  = d_latlon["lat"]
last_trips["dest_lon"]  = d_latlon["lon"]

# drop decode failures before sjoin
first_trips = first_trips[first_trips["orig_lat"].notna() & first_trips["orig_lon"].notna()]
last_trips  = last_trips[last_trips["dest_lat"].notna() & last_trips["dest_lon"].notna()]


# =========================
# 3️⃣ TRIP-LEVEL OD FILTER (CORRECT, OPTIMIZED)
# =========================

# origin check (first trip only)
gdf_o = gpd.GeoDataFrame(
    first_trips,
    geometry=gpd.points_from_xy(first_trips["orig_lon"], first_trips["orig_lat"]),
    crs="EPSG:4326"
)
gdf_o = gpd.sjoin(gdf_o, orig_poly, predicate="within", how="inner")
valid_o_ids = set(gdf_o["linked_trip_id"])

# destination check (last trip only)
gdf_d = gpd.GeoDataFrame(
    last_trips,
    geometry=gpd.points_from_xy(last_trips["dest_lon"], last_trips["dest_lat"]),
    crs="EPSG:4326"
)
gdf_d = gpd.sjoin(gdf_d, dest_poly, predicate="within", how="inner")
valid_d_ids = set(gdf_d["linked_trip_id"])

valid_linked_ids = valid_o_ids & valid_d_ids

# filter full df to keep all segments (same logic), then compute coords only for filtered data
gdf = df[df["linked_trip_id"].isin(valid_linked_ids)].copy()

print("After OD (trip-level) filter, rows:", len(gdf))
print("Unique linked trips:", gdf["linked_trip_id"].nunique())

if gdf.empty:
    stats = {
        "schema": "nova.complete_trip.od_stats.v1",
        "generated_at": datetime.utcnow().isoformat() + "Z",
        "od": {"origin": ORIG_TRACT, "destination": DEST_TRACT},
        "coverage": {"temporal": "year-2020", "spatial": "Salt Lake 6-county"},
        "counts": {"linked_trips": 0},
        "note": "No linked trips after trip-level OD filter"
    }
    with open(OUTPUT_STATS_JSON, "w", encoding="utf-8") as f:
        json.dump(stats, f, indent=2, allow_nan=False)
    raise SystemExit


# =========================
# 4️⃣ (UNCHANGED LOGIC) OPTIONAL: decode all points for filtered data only
# (kept to preserve your existing columns if you later reuse them)
# =========================

o_latlon_all = _decode_unique_geohash(gdf["geohash7_orig"])
d_latlon_all = _decode_unique_geohash(gdf["geohash7_dest"])

gdf["orig_lat"] = o_latlon_all["lat"]
gdf["orig_lon"] = o_latlon_all["lon"]
gdf["dest_lat"] = d_latlon_all["lat"]
gdf["dest_lon"] = d_latlon_all["lon"]

gdf = gdf[
    gdf["orig_lat"].notna() & gdf["orig_lon"].notna() &
    gdf["dest_lat"].notna() & gdf["dest_lon"].notna()
]


# =========================
# 5️⃣ DURATION
# =========================

gdf["duration_min"] = (
    gdf["local_datetime_end"] - gdf["local_datetime_start"]
).dt.total_seconds() / 60


# =========================
# 6️⃣ AGGREGATION
# =========================

trip_stats = (
    gdf
    .groupby("linked_trip_id")
    .agg(
        total_duration=("duration_min", "sum"),
        segments=("duration_min", "size"),
        modes=("travel_mode", lambda x: set(m.lower().strip() for m in x))
    )
)


# =========================
# 7️⃣ FINAL STATS
# =========================

dur = trip_stats["total_duration"].to_numpy()
segments = trip_stats["segments"].to_numpy()
modes = trip_stats["modes"].to_numpy()

def pct(a, q):
    return float(np.percentile(a, q))

BIN_WIDTH = 5
MAX_TIME = 180

bins = np.arange(0, MAX_TIME + BIN_WIDTH, BIN_WIDTH)
dur_capped = np.clip(dur, 0, MAX_TIME)
hist_counts, bin_edges = np.histogram(dur_capped, bins=bins)

travel_time_hist = {
    "bin_width_min": BIN_WIDTH,
    "max_time_min": MAX_TIME,
    "bin_edges_min": bin_edges.tolist(),
    "counts": hist_counts.tolist()
}

stats = {
    "schema": "nova.complete_trip.od_stats.v1",
    "generated_at": datetime.utcnow().isoformat() + "Z",
    "od": {"origin": ORIG_TRACT, "destination": DEST_TRACT},
    "coverage": {"temporal": "year-2020", "spatial": "Salt Lake 6-county"},
    "counts": {"linked_trips": int(len(dur))},
    "trip_duration_min": {
        "min": float(dur.min()),
        "mean": float(dur.mean()),
        "p25": pct(dur, 25),
        "median": pct(dur, 50),
        "p75": pct(dur, 75),
        "max": float(dur.max())
    },
    "segments": {
        "avg": float(segments.mean()),
        "p75": int(pct(segments, 75)),
        "max": int(segments.max())
    },
    "mode_involvement": {
        "car": float(sum("car" in m for m in modes) / len(modes)),
        "bus": float(sum("bus" in m for m in modes) / len(modes)),
        "rail": float(sum("rail" in m for m in modes) / len(modes)),
        "walk": float(sum("walk" in m for m in modes) / len(modes))
    },
    "travel_time_distribution": travel_time_hist
}

with open(OUTPUT_STATS_JSON, "w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, allow_nan=False)

print("Stats JSON written to:", OUTPUT_STATS_JSON)


In [21]:
print(hist_counts)


[ 283 2034  987  229   76   24   13   11    5    4    7    4    3    3
    1    2    1   12]
